# Predict Next Week Sales (Rolling Week)
## AdventureWorks Example
Using the [AdventureWorks Database](https://improveandrepeat.com/2019/02/use-the-adventureworks-sample-database-for-your-examples/https://improveandrepeat.com/2019/02/use-the-adventureworks-sample-database-for-your-examples/) as an example

I wont even demonstrate the predictive modeling phase - this is a fake database and I doubt there are any predictive signals built into it.

### Imports

In [2]:
### Standard Imports - Sorry PEP8 fans, do not look below
import pandas as pd, numpy as np, os, re, json, sys
from pathlib import Path
from datetime import datetime

## Specific Imports

# If you are not a Rasgo customer, open source version RasgoQL is available for free
# I am using Rasgo Enterprise
import pyrasgo
raskey = os.getenv('RASGOMAIN')
rasgo = pyrasgo.connect(raskey)

### Display options for notebooks
pd_options = {'display.max_rows': 25,'display.max_columns': 500}
[pd.set_option(option, setting) for option, setting in pd_options.items()]

### set path directories
curr_dir = Path(os.getcwd())
data_dir = Path(curr_dir.parents[0] / 'data/')
artifacts_dir = Path(curr_dir.parents[0] / 'artifacts/')

## Setup a Feature Store

In [3]:
sales = rasgo.get.dataset(fqtn="ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES")
prod = rasgo.get.dataset(fqtn="ADVENTUREWORKS.PUBLIC.DIMPRODUCT")
cust = rasgo.get.dataset(fqtn="ADVENTUREWORKS.PUBLIC.DIMCUSTOMER")
geo = rasgo.get.dataset(fqtn="ADVENTUREWORKS.PUBLIC.DIMGEOGRAPHY")

# aggregate sales by date
daily = sales.aggregate(group_by=['ORDERDATE'], aggregations={'SALESAMOUNT':['SUM']}).rename(renames={'SALESAMOUNT_SUM':'SALESAMOUNT'})
daily = daily.datespine(date_col='ORDERDATE',start_timestamp='2010-12-29',end_timestamp='2014-01-28',interval_type='DAY')
daily = daily.drop_columns(exclude_cols=['ORDERDATE','ORDERDATE_SPINE_END'])
daily = daily.rename(renames={'ORDERDATE_SPINE_START':'ORDERDATE'})

# pivot sales by product and date
sales_prod = sales.join(join_table=prod, join_columns={'PRODUCTKEY':'PRODUCTKEY'}, join_type='INNER')
prod_pvt = sales_prod.pivot(dimensions=['ORDERDATE'], pivot_column='SALESAMOUNT', value_column='PRODUCTLINE', agg_method='SUM', list_of_vals=['R','M'])

# aggregate, pivot, and lag by geo
sales_cust = sales.join(join_table=cust, join_columns={'CUSTOMERKEY':'CUSTOMERKEY'}, join_type='INNER')
sales_geo = sales_cust.join(join_table=geo, join_columns={'GEOGRAPHYKEY':'GEOGRAPHYKEY'}, join_type='INNER')
sales_geo = sales_geo.aggregate(group_by=['ORDERDATE','ENGLISHCOUNTRYREGIONNAME'], aggregations={'SALESAMOUNT':['SUM']}).rename(renames={'SALESAMOUNT_SUM':'SALESAMOUNT'})
sales_geo = sales_geo.pivot(dimensions=['ORDERDATE'], pivot_column='SALESAMOUNT', value_column='ENGLISHCOUNTRYREGIONNAME', agg_method='SUM')

# bring it all together
merged = daily.join(join_table = prod_pvt, join_columns={'ORDERDATE':'ORDERDATE'}, join_type='LEFT', join_prefix='PRODUCT_SALES')
merged = merged.join(join_table = sales_geo, join_columns={'ORDERDATE':'ORDERDATE'}, join_type='LEFT', join_prefix='COUNTRY_SALES')
merged = merged.drop_columns(exclude_cols=['PRODUCT_SALES_ORDERDATE','COUNTRY_SALES_ORDERDATE'])

# Moving Window aggs
movingaggs = merged.rolling_agg(aggregations={'SALESAMOUNT': ['MIN', 'MAX', 'SUM', 'AVG','STDDEV'],
                                            'PRODUCT_SALES_R': ['MIN', 'AVG'],
                                            'PRODUCT_SALES_M': ['AVG'],
                                            'COUNTRY_SALES_AUSTRALIA': ['AVG'],
                                            'COUNTRY_SALES_UNITED_STATES': ['AVG','STDDEV'],
                                            'COUNTRY_SALES_UNITED_KINGDOM': ['AVG'],
                                            'COUNTRY_SALES_GERMANY': ['AVG']},
                              offsets=[3, 7, 14, 30],
                              order_by=["ORDERDATE"])

# Lag Variables
lags = movingaggs.lag(columns=['SUM_SALESAMOUNT_7',
                               'SUM_SALESAMOUNT_14',
                               'SUM_SALESAMOUNT_30',
                               'AVG_SALESAMOUNT_7',
                               'AVG_SALESAMOUNT_14',
                               'AVG_SALESAMOUNT_30',
                               'AVG_PRODUCT_SALES_R_7'],
                      amounts=[7, 14, 21, 28],
                      order_by=['ORDERDATE'],
                      partition=['NULL'])

# some creative ones
creative = lags.math(math_ops=[
    """(5*SUM_SALESAMOUNT_7 + 4*LAG_SUM_SALESAMOUNT_7_7 + 
    3*LAG_SUM_SALESAMOUNT_7_14 + 2*LAG_SUM_SALESAMOUNT_7_21 + 
    LAG_SUM_SALESAMOUNT_7_28)/15""",
    '(AVG_PRODUCT_SALES_R_7 - LAG_AVG_PRODUCT_SALES_R_7_21)/21',
    '(AVG_SALESAMOUNT_14 - LAG_AVG_SALESAMOUNT_14_28)/28',
    '(AVG_SALESAMOUNT_7 - LAG_AVG_SALESAMOUNT_7_14)/14',
    'AVG_SALESAMOUNT_7/NULLIF(AVG_SALESAMOUNT_30, 0)',
    'AVG_COUNTRY_SALES_UNITED_STATES_3/NULLIF(AVG_COUNTRY_SALES_UNITED_STATES_30, 0)',
    'AVG_PRODUCT_SALES_M_3/NULLIF(AVG_PRODUCT_SALES_M_30, 0)'],
    names=['MOVING_AVERAGE_SUM_SALESAMOUNT',
           'PRODUCT_SALES_R_AVG_7_RATE_OF_CHANGE_21',
           'AVG_SALESAMOUNT_14_RATE_OF_CHANGE_28',
           'AVG_SALESAMOUNT_7_RATE_OF_CHANGE_14',
           'AVG_SALESAMOUNT_7_AVG_30_RATIO',
           'COUNTRY_SALES_UNITED_STATES_AVG_3_RATIO',
           'PRODUCT_SALES_M_AVG_3_RATIO'])


creative.preview()

,ORDERDATE,SALESAMOUNT,PRODUCT_SALES_R,PRODUCT_SALES_M,COUNTRY_SALES_CANADA,COUNTRY_SALES_UNITED_STATES,COUNTRY_SALES_AUSTRALIA,COUNTRY_SALES_UNITED_KINGDOM,COUNTRY_SALES_GERMANY,COUNTRY_SALES_FRANCE,MIN_SALESAMOUNT_3,MIN_SALESAMOUNT_7,MIN_SALESAMOUNT_14,MIN_SALESAMOUNT_30,MAX_SALESAMOUNT_3,MAX_SALESAMOUNT_7,MAX_SALESAMOUNT_14,MAX_SALESAMOUNT_30,SUM_SALESAMOUNT_3,SUM_SALESAMOUNT_7,SUM_SALESAMOUNT_14,SUM_SALESAMOUNT_30,AVG_SALESAMOUNT_3,AVG_SALESAMOUNT_7,AVG_SALESAMOUNT_14,AVG_SALESAMOUNT_30,STDDEV_SALESAMOUNT_3,STDDEV_SALESAMOUNT_7,STDDEV_SALESAMOUNT_14,STDDEV_SALESAMOUNT_30,MIN_PRODUCT_SALES_R_3,MIN_PRODUCT_SALES_R_7,MIN_PRODUCT_SALES_R_14,MIN_PRODUCT_SALES_R_30,AVG_PRODUCT_SALES_R_3,AVG_PRODUCT_SALES_R_7,AVG_PRODUCT_SALES_R_14,AVG_PRODUCT_SALES_R_30,AVG_PRODUCT_SALES_M_3,AVG_PRODUCT_SALES_M_7,AVG_PRODUCT_SALES_M_14,AVG_PRODUCT_SALES_M_30,AVG_COUNTRY_SALES_AUSTRALIA_3,AVG_COUNTRY_SALES_AUSTRALIA_7,AVG_COUNTRY_SALES_AUSTRALIA_14,AVG_COUNTRY_SALES_AUSTRALIA_30,AVG_COUNTRY_SALES_UNITED_STATES_3,AVG_COUNTRY_SALES_UNITED_STATES_7,AVG_COUNTRY_SALES_UNITED_STATES_14,AVG_COUNTRY_SALES_UNITED_STATES_30,STDDEV_COUNTRY_SALES_UNITED_STATES_3,STDDEV_COUNTRY_SALES_UNITED_STATES_7,STDDEV_COUNTRY_SALES_UNITED_STATES_14,STDDEV_COUNTRY_SALES_UNITED_STATES_30,AVG_COUNTRY_SALES_UNITED_KINGDOM_3,AVG_COUNTRY_SALES_UNITED_KINGDOM_7,AVG_COUNTRY_SALES_UNITED_KINGDOM_14,AVG_COUNTRY_SALES_UNITED_KINGDOM_30,AVG_COUNTRY_SALES_GERMANY_3,AVG_COUNTRY_SALES_GERMANY_7,AVG_COUNTRY_SALES_GERMANY_14,AVG_COUNTRY_SALES_GERMANY_30,LAG_SUM_SALESAMOUNT_7_7,LAG_SUM_SALESAMOUNT_7_14,LAG_SUM_SALESAMOUNT_7_21,LAG_SUM_SALESAMOUNT_7_28,LAG_SUM_SALESAMOUNT_14_7,LAG_SUM_SALESAMOUNT_14_14,LAG_SUM_SALESAMOUNT_14_21,LAG_SUM_SALESAMOUNT_14_28,LAG_SUM_SALESAMOUNT_30_7,LAG_SUM_SALESAMOUNT_30_14,LAG_SUM_SALESAMOUNT_30_21,LAG_SUM_SALESAMOUNT_30_28,LAG_AVG_SALESAMOUNT_7_7,LAG_AVG_SALESAMOUNT_7_14,LAG_AVG_SALESAMOUNT_7_21,LAG_AVG_SALESAMOUNT_7_28,LAG_AVG_SALESAMOUNT_14_7,LAG_AVG_SALESAMOUNT_14_14,LAG_AVG_SALESAMOUNT_14_21,LAG_AVG_SALESAMOUNT_14_28,LAG_AVG_SALESAMOUNT_30_7,LAG_AVG_SALESAMOUNT_30_14,LAG_AVG_SALESAMOUNT_30_21,LAG_AVG_SALESAMOUNT_30_28,LAG_AVG_PRODUCT_SALES_R_7_7,LAG_AVG_PRODUCT_SALES_R_7_14,LAG_AVG_PRODUCT_SALES_R_7_21,LAG_AVG_PRODUCT_SALES_R_7_28,MOVING_AVERAGE_SUM_SALESAMOUNT,PRODUCT_SALES_R_AVG_7_RATE_OF_CHANGE_21,AVG_SALESAMOUNT_14_RATE_OF_CHANGE_28,AVG_SALESAMOUNT_7_RATE_OF_CHANGE_14,AVG_SALESAMOUNT_7_AVG_30_RATIO,COUNTRY_SALES_UNITED_STATES_AVG_3_RATIO,PRODUCT_SALES_M_AVG_3_RATIO
0,2010-12-29,14477.3382,4277.3682,10199.9700,3578.2700,4099.0882,3399.9900,None,None,3399.9900,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382,14477.3382000,14477.3382000,14477.3382000,14477.3382000,NaN,NaN,NaN,NaN,4277.3682,4277.3682,4277.3682,4277.3682,4277.3682000,4277.3682000,4277.3682000,4277.3682000,10199.9700000,10199.9700000,10199.9700000,10199.9700000,3399.9900000,3399.9900000,3399.9900000,3399.9900000,4099.0882000,4099.0882000,4099.0882000,4099.0882000,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.000000000000,1.000000000000,1.000000000000
1,2010-12-30,13931.5200,7156.5400,6774.9800,None,3578.2700,10353.2500,None,None,None,13931.5200,13931.5200,13931.5200,13931.5200,14477.3382,14477.3382,14477.3382,14477.3382,28408.8582,28408.8582,28408.8582,28408.8582,14204.4291000,14204.4291000,14204.4291000,14204.4291000,385.951751,385.951751,385.951751,385.951751,4277.3682,4277.3682,4277.3682,4277.3682,5716.9541000,5716.9541000,5716.9541000,5716.9541000,8487.4750000,8487.4750000,8487.4750000,8487.4750000,6876.6200000,6876.6200000,6876.6200000,6876.6200000,3838.6791000,3838.6791000,3838.6791000,3838.6791000,368.274081,368.274081,368.274081,368.274081,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

In [ ]:
rasgo.publish.dataset(dataset=creative, name="Sales by Day FeatureStore", resource_key='adventureworks_sales_by_day_featurestore')

__________________________________
## Create Target Variable for Model Training

We're imagining a scenario where the feature store above was already created. Now, someone comes along to build a model. That's why we start over from scratch (imagine a scenario where the target definiton might not be possible with only what is in the feature store)

In [4]:
sales = rasgo.get.dataset(fqtn="ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES")

# aggregate sales by date
daily = sales.aggregate(group_by=['ORDERDATE'], aggregations={'SALESAMOUNT':['SUM']}).rename(renames={'SALESAMOUNT_SUM':'SALESAMOUNT'})
daily = daily.datespine(date_col='ORDERDATE',start_timestamp='2010-12-29',end_timestamp='2014-01-28',interval_type='DAY')
daily = daily.drop_columns(exclude_cols=['ORDERDATE','ORDERDATE_SPINE_END'])
daily = daily.rename(renames={'ORDERDATE_SPINE_START':'ORDERDATE'})

# look forwards and sum the future sales
target = daily.rolling_agg(aggregations={'SALESAMOUNT': ['SUM']},
                              offsets=[-8],
                              order_by=["ORDERDATE"])

# filter out the oldest rows where we know theres not enough history for good features
# our max lag is 30
target = target.filter(filter_statements=["ORDERDATE > '2011-01-31'"])

# exclude current day sales and predict only future 7 days
target = target.math(math_ops=["SUM_SALESAMOUNT__8 - SALESAMOUNT"], names=['TARGET_SALES_NEXT_WEEK'])

# clean up the dataset
target=target.drop_columns(include_cols=['ORDERDATE','TARGET_SALES_NEXT_WEEK'])

# filter out the newest rows where we know a full target wasnt calculated
# also the data is messed up after 12/21
# our max lag is 30
target = target.filter(filter_statements=["ORDERDATE < '2013-12-21'"])

# impute zero or datarobot will ignore row
target = target.impute(imputations={'TARGET_SALES_NEXT_WEEK':0})

target.preview()

,ORDERDATE,TARGET_SALES_NEXT_WEEK
0,2013-12-20,507484.7000
1,2013-12-19,525878.7600
2,2013-12-18,548038.4400
3,2013-12-17,536407.3200
4,2013-12-16,550339.6800
5,2013-12-15,538893.5300
6,2013-12-14,532979.3200
7,2013-12-13,525521.1700
8,2013-12-12,542293.7700
9,2013-12-11,572135.9900


In [ ]:
rasgo.publish.dataset(dataset=target, name="Predict Future Sales - Target", resource_key='adventureworks_sales_by_day_targets')

## Create Training Dataset

In [5]:
target = rasgo.get.dataset(resource_key='adventureworks_sales_by_day_targets')
features = rasgo.get.dataset(resource_key='adventureworks_sales_by_day_featurestore')

training = target.join(join_table=features, join_type='LEFT', join_columns={'ORDERDATE':'ORDERDATE'})

training.preview()

,ORDERDATE,TARGET_SALES_NEXT_WEEK,SALESAMOUNT,PRODUCT_SALES_R,PRODUCT_SALES_M,COUNTRY_SALES_CANADA,COUNTRY_SALES_UNITED_STATES,COUNTRY_SALES_AUSTRALIA,COUNTRY_SALES_UNITED_KINGDOM,COUNTRY_SALES_GERMANY,COUNTRY_SALES_FRANCE,MIN_SALESAMOUNT_3,MIN_SALESAMOUNT_7,MIN_SALESAMOUNT_14,MIN_SALESAMOUNT_30,MAX_SALESAMOUNT_3,MAX_SALESAMOUNT_7,MAX_SALESAMOUNT_14,MAX_SALESAMOUNT_30,SUM_SALESAMOUNT_3,SUM_SALESAMOUNT_7,SUM_SALESAMOUNT_14,SUM_SALESAMOUNT_30,AVG_SALESAMOUNT_3,AVG_SALESAMOUNT_7,AVG_SALESAMOUNT_14,AVG_SALESAMOUNT_30,STDDEV_SALESAMOUNT_3,STDDEV_SALESAMOUNT_7,STDDEV_SALESAMOUNT_14,STDDEV_SALESAMOUNT_30,MIN_PRODUCT_SALES_R_3,MIN_PRODUCT_SALES_R_7,MIN_PRODUCT_SALES_R_14,MIN_PRODUCT_SALES_R_30,AVG_PRODUCT_SALES_R_3,AVG_PRODUCT_SALES_R_7,AVG_PRODUCT_SALES_R_14,AVG_PRODUCT_SALES_R_30,AVG_PRODUCT_SALES_M_3,AVG_PRODUCT_SALES_M_7,AVG_PRODUCT_SALES_M_14,AVG_PRODUCT_SALES_M_30,AVG_COUNTRY_SALES_AUSTRALIA_3,AVG_COUNTRY_SALES_AUSTRALIA_7,AVG_COUNTRY_SALES_AUSTRALIA_14,AVG_COUNTRY_SALES_AUSTRALIA_30,AVG_COUNTRY_SALES_UNITED_STATES_3,AVG_COUNTRY_SALES_UNITED_STATES_7,AVG_COUNTRY_SALES_UNITED_STATES_14,AVG_COUNTRY_SALES_UNITED_STATES_30,STDDEV_COUNTRY_SALES_UNITED_STATES_3,STDDEV_COUNTRY_SALES_UNITED_STATES_7,STDDEV_COUNTRY_SALES_UNITED_STATES_14,STDDEV_COUNTRY_SALES_UNITED_STATES_30,AVG_COUNTRY_SALES_UNITED_KINGDOM_3,AVG_COUNTRY_SALES_UNITED_KINGDOM_7,AVG_COUNTRY_SALES_UNITED_KINGDOM_14,AVG_COUNTRY_SALES_UNITED_KINGDOM_30,AVG_COUNTRY_SALES_GERMANY_3,AVG_COUNTRY_SALES_GERMANY_7,AVG_COUNTRY_SALES_GERMANY_14,AVG_COUNTRY_SALES_GERMANY_30,LAG_SUM_SALESAMOUNT_7_7,LAG_SUM_SALESAMOUNT_7_14,LAG_SUM_SALESAMOUNT_7_21,LAG_SUM_SALESAMOUNT_7_28,LAG_SUM_SALESAMOUNT_14_7,LAG_SUM_SALESAMOUNT_14_14,LAG_SUM_SALESAMOUNT_14_21,LAG_SUM_SALESAMOUNT_14_28,LAG_SUM_SALESAMOUNT_30_7,LAG_SUM_SALESAMOUNT_30_14,LAG_SUM_SALESAMOUNT_30_21,LAG_SUM_SALESAMOUNT_30_28,LAG_AVG_SALESAMOUNT_7_7,LAG_AVG_SALESAMOUNT_7_14,LAG_AVG_SALESAMOUNT_7_21,LAG_AVG_SALESAMOUNT_7_28,LAG_AVG_SALESAMOUNT_14_7,LAG_AVG_SALESAMOUNT_14_14,LAG_AVG_SALESAMOUNT_14_21,LAG_AVG_SALESAMOUNT_14_28,LAG_AVG_SALESAMOUNT_30_7,LAG_AVG_SALESAMOUNT_30_14,LAG_AVG_SALESAMOUNT_30_21,LAG_AVG_SALESAMOUNT_30_28,LAG_AVG_PRODUCT_SALES_R_7_7,LAG_AVG_PRODUCT_SALES_R_7_14,LAG_AVG_PRODUCT_SALES_R_7_21,LAG_AVG_PRODUCT_SALES_R_7_28,MOVING_AVERAGE_SUM_SALESAMOUNT,PRODUCT_SALES_R_AVG_7_RATE_OF_CHANGE_21,AVG_SALESAMOUNT_14_RATE_OF_CHANGE_28,AVG_SALESAMOUNT_7_RATE_OF_CHANGE_14,AVG_SALESAMOUNT_7_AVG_30_RATIO,COUNTRY_SALES_UNITED_STATES_AVG_3_RATIO,PRODUCT_SALES_M_AVG_3_RATIO
0,2011-02-04,132967.0446,14313.0800,14313.0800,None,None,7156.5400,7156.5400,None,None,None,14313.0800,10734.8100,10734.8100,6953.2600,25390.4282,25390.4282,28041.3200,38241.2900,72949.5746,127818.5410,247439.1420,498436.0530,18237.3936500,15977.3176250,16495.9428000,16078.5823548,4947.762496,4605.630040,5074.086828,6767.302405,10734.8100,7855.6382,4277.3682,3578.2700,14837.4036500,13855.4488750,13109.2861333,12689.0762258,6799.9800000,5658.3166666,6349.9812500,6180.8641176,12520.4395500,9005.8995500,8692.6710428,8192.6047600,3753.0445500,4189.3226000,5051.6422363,5491.6036818,2643.952967,2283.417237,3228.216766,3329.209598,2138.6841000,2138.6841000,2618.5460666,2758.3540545,3578.2700000,3578.2700000,3578.2700000,3560.4420000,134632.7792,136213.7546,110887.3482,98671.5628,260111.7238,232966.3028,199002.3810,127080.4210,473388.1630,349490.1938,227411.2392,127080.4210,16829.0974000,17026.7193250,13860.9185250,12333.9453500,17340.7815866,15531.0868533,13266.8254000,12708.0421000,15270.5859032,14562.0914083,13377.1317176,12708.0421000,12600.9849000,12786.1068250,12164.0485250,10637.0753500,127114.0896533333,80.542873809524,135.282167857143,-74.957264285714,0.993701886922,0.683415040025,1.100166557721
1,2011-02-10,126662.8828,11929.7264,8554.7364,3374.9900,None,4277.3682,7652.3582,None,None,None,11929.7264,11255.6282,10734.8100,6953.2600,21469.6200,25390.4282,25390.4282,38241.2900,72378.7564,137244.4128,239761.7838,521050.5894,18094.6891000,17155.5516000,15984.1189200,168

In [ ]:
rasgo.publish.dataset(dataset=training, name="Predict Future Sales - Training Data", resource_key='adventureworks_sales_by_day_training', attributes={'target':'TARGET_SALES_NEXT_WEEK'})

### Export data as csv

In [6]:
target = rasgo.get.dataset(resource_key='adventureworks_sales_by_day_targets')
features = rasgo.get.dataset(resource_key='adventureworks_sales_by_day_featurestore')
training = rasgo.get.dataset(resource_key='adventureworks_sales_by_day_training')

In [8]:
dfTarget = target.to_df()
dfFeatures = features.to_df()
dfTraining = training.to_df()

In [9]:
dfTarget.to_csv(Path(data_dir) / 'target.csv', index=False, chunksize=2000)
dfFeatures.to_csv(Path(data_dir) / 'features.csv', index=False, chunksize=2000)
dfTraining.to_csv(Path(data_dir) / 'training.csv', index=False, chunksize=2000)